In [60]:

import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

sentences_test = ["Don't remove redo option from the page, I think it is useful","Don't remove redo option from the page, I also think it is useful","I don't think it is useful to have redo option, reomove from the page"]
with tf.Session() as session:

  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  sentences_embed = session.run(embed(sentences_test))

# For evaluation we use exactly normalized rather than
# approximately normalized.
sts_encode1 = tf.nn.l2_normalize(sentences_embed[0])
sts_encode2 = tf.nn.l2_normalize(sentences_embed[1])
cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2))
clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
sim_scores = 1.0 - tf.acos(clip_cosine_similarities)
with tf.Session() as sess:    
  print("Sentence 1 and 2 Score :")
  print(sim_scores.eval())  

#sentence 1 and 3
sts_encode1 = tf.nn.l2_normalize(sentences_embed[0])
sts_encode2 = tf.nn.l2_normalize(sentences_embed[2])
cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2))
clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
sim_scores = 1.0 - tf.acos(clip_cosine_similarities)
with tf.Session() as sess:  
  print("Sentence 1 and 3 Score :")
  print(sim_scores.eval()) 

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Sentence 1 and 2 Score :
0.89120203
Sentence 1 and 3 Score :
0.5125967


0.89120203


0.5125967


In [30]:
print(np.inner(sentences_embed[0], sentences_embed[1]))
print(np.inner(sentences_embed[1], sentences_embed[2]))
print(np.inner(sentences_embed[0], sentences_embed[2]))

0.99408734
0.88090116
0.8835519


In [36]:
from scipy.spatial import distance
print(distance.euclidean(sentences_embed[0], sentences_embed[1]))
print(distance.euclidean(sentences_embed[1], sentences_embed[2]))
print(distance.euclidean(sentences_embed[0], sentences_embed[2]))

0.10874460637569427
0.488055020570755
0.48259320855140686


In [61]:
from sklearn.feature_extraction.text import CountVectorizer

# list of text documents
sentences_test_SVM = ["Don't remove redo option from the page, I think it is useful","Don't remove redo option from the page, I also think it is useful","I don't think it is useful to have redo option, reomove from the page"]
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(sentences_test)
# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(sentences_test)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

print("Sentence 1 and 2 Score :")
print(cosine_similarity(vector[0].toarray(), vector[1].toarray()))
print("Sentence 1 and 3 Score :")
print(cosine_similarity(vector[0].toarray(), vector[2].toarray()))

{'don': 1, 'remove': 9, 'redo': 8, 'option': 6, 'from': 2, 'the': 11, 'page': 7, 'think': 12, 'it': 5, 'is': 4, 'useful': 14, 'also': 0, 'to': 13, 'have': 3, 'reomove': 10}
(3, 15)
<class 'scipy.sparse.csr.csr_matrix'>
[[0 1 1 0 1 1 1 1 1 1 0 1 1 0 1]
 [1 1 1 0 1 1 1 1 1 1 0 1 1 0 1]
 [0 1 1 1 1 1 1 1 1 0 1 1 1 1 1]]
Sentence 1 and 2 Score :
[[0.95742711]]
Sentence 1 and 3 Score :
[[0.83624201]]


In [23]:
import spacy

nlp = spacy.load('en')

#search_doc = nlp("Add brace new line code style for switch expressions")
#main_doc = nlp("New line setting for C# switch expression is missing")

search_doc = nlp("Don''t remove redo option from the page, I think it is useful",)
main_doc = nlp("I don't think it is useful to have redo option, reomove from the page")
print(main_doc.similarity(search_doc))

0.8026810906443756


/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
